In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\brain\\TUMOR-DETECTION-MLOPS-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\brain\\TUMOR-DETECTION-MLOPS-'

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [15]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [17]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [20]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    layers = None


    
    def get_base_model(self):
        self.model = tf.keras.applications.resnet50.ResNet50(
            input_tensor= tf.keras.layers.Input(shape=(256, 256, 3)),
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, learning_rate):
        for layer in model.layers:
            layer.trainable = False

        # Add classification head to the base model
        headmodel = model.output
        headmodel = tf.keras.layers.AveragePooling2D(pool_size=(4, 4))(headmodel)
        headmodel = tf.keras.layers.Flatten(name='flatten')(headmodel)
        headmodel = tf.keras.layers.Dense(256, activation="relu")(headmodel)
        headmodel = tf.keras.layers.Dropout(0.3)(headmodel)
        headmodel = tf.keras.layers.Dense(256, activation="relu")(headmodel)
        headmodel = tf.keras.layers.Dropout(0.3)(headmodel)
        #headmodel = tf.keras.layers.Dense(256, activation="relu")(headmodel)
        #headmodel = tf.keras.layers.Dropout(0.3)(headmodel)
        headmodel = tf.keras.layers.Dense(units = classes, activation='softmax')(headmodel)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=headmodel)


        full_model.compile(
            loss = 'categorical_crossentropy',
              optimizer='adam', 
              metrics= ["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    


In [21]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-07-10 18:09:53,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-10 18:09:53,057: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-10 18:09:53,065: INFO: common: created directory at: artifacts]
[2023-07-10 18:09:53,073: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-07-10 18:10:04,712: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


c:\Users\User\anaconda3\envs\brain\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          